In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
from scipy.stats import pearsonr
import pandas as pd
import csv
import os

In [ ]:
####### Simulation parameters #######

N = 200                                   # Number of neurons
N2 = int(N/2)

#Synaptic connections
p =  0.3                                 # Probability of non-zero elements in the weight matrix
gsyn = 0.5                               # Initial synaptic strength

alpha = 0.25                              # Weight regularization parameter (training)

#Dynamics
dt = 0.1                                  # Time step (time scale 10 ms)
itmax = 1000                              # Number of iterations (1000 -> 1 sec)
sigman = 1                                # Noise standard deviation -> noise in the dynamics


#Stimulus
itstim = 200                              # stimulation time
amp_current = 20                          # stimulus intensity
amp0 = 4                                  # changed from amp0=8 to amp0=4, in order to have the same current amplitudes as in the pre-training case for both oscillations and sequences


#TRAINING
ftrain = 1                               # fraction of neurons to train
nloop  = 16                              # number of loops, 0 pre-training, last: post-training.
nloop_train = 10                         # last training loop

cant_seed = 2                            # independent simulations (for generating ensemble), is changed in learning loop


ts= 5
b = 1 / ts

iout = np.arange(0, N)  # neurons to observe outputs

### Folder organization

A separate folder `simulation_{num_simulation}` is created for each simulation, corresponding to a different resting potential value \(V_r\).

Within each folder, the following subdirectories are generated to organize the simulation outputs:

- **`simulation_{num_simulation}_currents/`** → contains the input currents (external, noise, and synaptic).  
- **`simulation_{num_simulation}_inputs/`** → stores the total input currents received by each neuron (sum of all current components). One file per training/testing loop.
- **`simulation_{num_simulation}_connectivity_matrix/`** → contains the synaptic weight matrices for the pre-training (nloop=0) and after training (nloop = 11) loops.
- **`simulation_{num_simulation}_outputs/`** → stores the network output activity of each neuron. One file per training/testing loop.
- **`simulation_{num_simulation}_nspikes/`** → contains the spike counts for each neuron. One file per training/testing loop.

A parameter file named **`simulation_{num_simulation}_parameters.csv`** is also created inside each main folder, storing all relevant simulation parameters (e.g., \(N, p, g_{syn}, dt, \alpha, V_t, b, V_r\)) for reproducibility.


In [3]:
####### File organization functions #######

def create_subfolder(parent_folder, subfolder_name):
    path = os.path.join(parent_folder, subfolder_name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path

def create_folders(num_simulation): 
    # Carpeta principal de la simulación
    folder_name = f"simulation_{num_simulation}"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Subcarpetas dentro de la simulation

    sub_pesos = create_subfolder(folder_name, f"simulation_{num_simulation}_connectivity_matrix")
    sub_currents = create_subfolder(folder_name, f"simulation_{num_simulation}_currents")
    sub_inputs = create_subfolder(folder_name, f"simulation_{num_simulation}_inputs")
    sub_outputs = create_subfolder(folder_name, f"simulation_{num_simulation}_outputs")
    sub_nspikes = create_subfolder(folder_name, f"simulation_{num_simulation}_nspikes")

    return folder_name, sub_pesos, sub_currents, sub_inputs, sub_outputs, sub_nspikes


def create_parameter_file(filename_results, num_simulation, folder_name, b, vt, vrest):
 #file donde guardo los parámetros de la simulación
    data_parametros = {
        'N': [N],
        'p': [p],
        'gsyn': [gsyn],
        'nloop': [nloop],
        'nloop_train':[nloop_train],
        'cant_seed': [cant_seed],
        'dt': [dt],
        'itmax': [itmax],
        'itstim': [itstim],
        'amp_current': [amp_current],
        'amp0': [amp0],
        'ftrain': [ftrain],
        'alpha': [alpha],
        'sigman': [sigman],
        'vt': [vt],
        'b': [b],
        'vrest': [vrest],
        'results_file': [filename_results],
    }


    df = pd.DataFrame(data_parametros)
    filename_parametros = f'simulation_{num_simulation}_parametros.csv'
    csv_parametros_path = os.path.join(folder_name, filename_parametros)
    df.to_csv(csv_parametros_path, index=False)

Generate oscillatory target

In [4]:
def generate_target(romega1, romega2, amp0):

    target=np.zeros((N,itmax)) 
    amp=np.random.uniform(size=N)*amp0 # Oscillation amplitude sampled uniformly between 0 and amp0

    phase=np.random.uniform(0,2*np.pi,size=N) # Random phase between 0 and 2pi

    index = [i for i in range(N)]
    indexs = np.random.permutation(index) 

    # We assign half of the neurons to frequency romega1 and the other half to romega2
    # To replicate that, in MEC there are mainly two oscillatory frequencies: theta and gamma 
    # Gamma is roughly five times higher in frequency than theta, so we set romega1 = 1 and romega2 = 5

    romega_vec = np.zeros(N)

    # Assign frequencies to neurons, randomly shuffled
    for i in range(N2):

        romega_vec[indexs[i]]= romega1
        romega_vec[indexs[i+N2]]=romega2

    
    omega=romega_vec*2*np.pi/itmax

    for it in range(itmax):
        target[:,it]=amp*np.cos(it*omega+phase) #Oscillatory target


            
    return target, amp, phase, omega, romega_vec, amp0


def save_target(seed, target, phase, omega, romega_vec, amp, amp0, num_simulation, folder_name, pqif):
    # Save target parameters  
    data = {'Neuron': range(N), 'Phase': phase, 'Frecuency': omega, 'romega': romega_vec, 'Amplitude': amp, 'amp0': amp0}
    df = pd.DataFrame(data)
    file_name = f'simulation_{num_simulation}_targets_parameters.csv'
    csv_target_path = os.path.join(folder_name, file_name)
    df.to_csv(csv_target_path, index=False)

    # Save target matrix
    target_df = pd.DataFrame(target.T, columns=[f'Neuron_{i}' for i in range(N)])
    file_name_target = f'simulation_{num_simulation}_targets_{pqif}_seed{seed}.csv'
    csv_target_path = os.path.join(folder_name, file_name_target)
    target_df.to_csv(csv_target_path, index=False)




def save_matrix_csv(matrix, file_name):
    with open(file_name, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        for row in matrix:
            row_list = [str(element) for element in row.flat]
            csv_writer.writerow(row_list)


In [ ]:

####### Dynamics and learning functions #######

def dynamics(x_var,r_var,I_var,nqif, b):
    dx=np.zeros(N)
    # Noise currents
    I_noise_lif = np.random.randn(N - nqif)*sigman 
    I_noise_qif = np.random.randn(nqif)*sigman
    #LIF
    dx[nqif:] = -x_var[nqif:] + I_var[nqif:] + I_noise_lif
    #QIF
    dx[:nqif] = 1 - np.cos(x_var[:nqif]) + I_var[:nqif]*(1 + np.cos(x_var[:nqif])) + I_noise_qif
    dr = -b*r_var
    return dx,dr


def detect(x, xnew, rnew, nspike, nqif, b, vt, vrest):
     # LIF neurons:
     # Neurons from index nqif onward are modeled as LIF.
     # They spike when the membrane potential crosses the threshold (vt) from below.
     ispike_lif = np.where(x[nqif:] < vt) and np.where(xnew[nqif:] > vt)
     ispike_lif = ispike_lif[0] + nqif
     if (len(ispike_lif) > 0):
         # After a spike, increase adaptation variable r by b
         rnew[ispike_lif[:]] = rnew[ispike_lif[:]] + b
         # Reset membrane potential to resting value
         xnew[ispike_lif[:]] = vrest
         # Count one more spike for each spiking neuron
         nspike[ispike_lif[:]] = nspike[ispike_lif[:]] + 1

     # QIF neurons:
     # Neurons from index 0 to nqif-1 are modeled as QIF.
     # The variable x represents a phase in [0, 2π).
     # A spike is detected when the phase increases by π or more between steps,
     # meaning the phase has advanced through the firing point.
     dpi = np.mod(np.pi - np.mod(x, 2 * np.pi), 2 * np.pi)  # distance to π
     ispike_qif = np.where((xnew[:nqif] - x[:nqif]) > 0) and np.where((xnew[:nqif] - x[:nqif] - dpi[:nqif]) > 0)
     if (len(ispike_qif) > 0):
         # After a spike, increase adaptation variable r by b
         rnew[ispike_qif[:]] = rnew[ispike_qif[:]] + b
         # Count one more spike for each spiking neuron
         nspike[ispike_qif[:]] = nspike[ispike_qif[:]] + 1

     return xnew, rnew, nspike



def evolution(x, r, Iext, w, nqif, it, dt, iout,nspike, b, vt, vrest):
    # Integration of the differential equations using RK2
    II = np.squeeze(np.asarray(Iext[:, it]))  
    v = w.dot(r.T).A1
    dx, dr = dynamics(x, r, II + v, nqif, b)
    xnew = x + dt * dx / 2
    rnew = r + dt * dr / 2
    dx, dr = dynamics(xnew, rnew, II + v, nqif, b)
    xnew = x + dt * dx
    rnew = r + dt * dr
    xnew, rnew, nspike = detect(x, xnew, rnew, nspike, nqif, b, vt, vrest)
    x, r = np.copy(xnew), np.copy(rnew)


    return x, r, nspike, r[iout], II, v


In [ ]:
def initialize_connectivity_matrix(N, p, gsyn):
    # Create a random N x N connectivity matrix with sparsity p
    # Values are drawn from a standard normal distribution
    w = sparse.random(N, N, p, data_rvs=np.random.randn).todense()
    
    # Remove autapses (no neuron connects to itself)
    np.fill_diagonal(w, 0)
    
    # Scale the weights by gsyn / sqrt(p * N) to normalize input magnitude
    w *= gsyn / np.sqrt(p * N)
    
    # Center each row so that the average outgoing weight is zero
    for i in range(N):
        i0 = np.where(w[i, :])[1]  # indices of non-zero outgoing weights
        if len(i0) > 0:
            av0 = np.sum(w[i, i0]) / len(i0)  # compute average of existing connections
            w[i, i0] -= av0  # subtract average to center the row

    return w


def initialize_neurons(N):
    x = np.random.uniform(size=N) * 2 * np.pi
    r = np.zeros(N)
    nspike = np.zeros(N)

    return x, r, nspike

def initialize_training(N, w):
    
    nind=np.zeros(N).astype('int')
    idx=[]
    P=[]
    for i in range(N):
        ind=np.where(w[i,:])[1]
        nind[i]=len(ind)
        idx.append(ind)
        P.append(np.identity(nind[i])/alpha)   
    return P, idx

def currents(N, itmax):
    Iext=np.zeros((N,itmax))
    Ibac=amp_current*(2*np.random.uniform(size=N)-1)
    Iext[:, :itstim] = Ibac[:, None]  # Vectorized assignment

    return Iext



def learning(it, iloop, w, r, P, idx, target, norm_w0, csv_writer):
    error = target[:, it:it + 1] - w @ r.reshape(N, 1)
    w1=np.zeros(w.shape)

    for i in range(N):
        ri = r[idx[i]].reshape(len(idx[i]), 1)
        k1 = P[i] @ ri
        k2 = ri.T @ P[i]
        den = 1 + ri.T @ k1
        P[i] -= (k1 @ k2) / den
        dw = error[i, 0] * P[i] @ r[idx[i]]
        w[i, idx[i]] += dw
    #     w1[i,idx[i]]+=dw   ## I commented out this part, because it takes a lot of time and can be uncommented when needed. Do same as here in sequences.


    # if it % 10 == 0:
    #     modt_value = (it-itstim) + (iloop-1) * (itmax-itstim)
    #     modw_value = np.log(np.linalg.norm(w1)/norm_w0)

        
    #     # Guardar los valores en el archivo CSV
    #     csv_writer.writerow([modt_value, modw_value])
    return w, P



Here we calculate the motifs. You can express them in terms of the components of the connectivity matrix.
I think it becomes easy to understand why we calculate the motifs this way once you write them as sums of the individual matrix elements. I attach the formulas for the reciprocal and convergent motifs. I recommend that you write the other two yourself

In [1]:
from IPython.display import Image, display

# display(Image(filename="motif_examples.png", width=500))

In [8]:
####### Motifs and dimensionality calculations #######
            
def motifs(w,gsyn,N):


    
    w=w-np.mean(w)
    
    ww=np.matmul(w,w)
    wtw=np.matmul(w.T,w)
    wwt=np.matmul(w,w.T)
    
    sigma2=np.trace(wwt)/N
    
    tau_rec=np.trace(ww)
    tau_rec/=sigma2*N
    
    tau_div=np.sum(wwt)-np.trace(wwt)
    tau_div/=sigma2*N*(N-1)
    
    tau_con=np.sum(wtw)-np.trace(wtw)
    tau_con/=sigma2*N*(N-1)
    
    tau_chn=2*(np.sum(ww)-np.trace(ww))
    tau_chn/=sigma2*N*(N-1)
    
    return sigma2,tau_rec,tau_div,tau_con,tau_chn


Here we calculate the dpr in terms of the diagonal and the non-diagonal components of the covariance matrix.

In [2]:
from IPython.display import Image, display

# display(Image(filename="covariance.png", width=500))
# display(Image(filename="dahmen_dpr_b.png", width=500)) # Mathematical suplementary of Dahmen et al. 2022

In [10]:
def dpr_bias(ccorr,N,nloop):
    a=np.extract(np.identity(N),ccorr)
    c=np.extract(1-np.identity(N),ccorr)
    am2=np.mean(a)**2
    astd2=np.var(a)*N/(N-1)
    cm2=np.mean(c)**2
    cstd2=np.var(c)*N*(N-1)/(N*(N-1)-2)
    
    astd_bias2=astd2*(nloop-1)/(nloop+1) -2*(am2-cm2)/(nloop-1)+ 2*cstd2/(nloop+1)
    cstd_bias2=(nloop-1)*cstd2/nloop - (am2-cm2)/nloop -4*(cm2-np.sqrt(am2*cm2))/(nloop*(N+1))
    
    dpr_bias=N/(1+(astd_bias2/am2)+(N-1)*((cstd_bias2/am2)+(cm2/am2)))
    
    return dpr_bias

In [ ]:
####### Main simulation loop #######


# Loop over different network compositions (LIF, QIF, and mixed)
for pqif in [0, 0.25, 0.5, 0.75, 1]:  # I changed to all pqif compositions here
    num_simulation = 0 # Initialized for creating one folder for each different vrest value

    if pqif==1:
        vt_vect = [None] # The theta model of QIF neurons has a fixed threshold at infinity
    else:
        vt_vect = [0]  # LIF


    for vt in vt_vect:
        if vt==0:
            vrest_vect = [-8.5, -12.3, -17, -22] # Included all four vrest values, one per simulation
        if vt == None:
            vrest_vect = [None, None, None, None] # The theta model of QIF neurons has a fixed resting potential at  -infinity, but four elements is included in list so there is one in each simulation

        
        for vrest in vrest_vect:
            num_simulation +=1
            #create folders
            nombre_carpeta, sub_pesos, sub_currents, sub_inputs, sub_outputs, sub_nspikes = create_folders(num_simulation)
            #save target parameters and matrix

            filename_results = f'simulation_{num_simulation}_results.csv'
            create_parameter_file(filename_results, num_simulation, nombre_carpeta, b, vt, vrest=vrest)

            # here we save the motif values
            csv_file_path = os.path.join(nombre_carpeta, filename_results)
            column_names = [ 'pqif' ,'seed','nloop', 'sigma2', 'tau_rec','tau_div','tau_con','tau_chn']
            cant_seed = 2 #sometimes we use more seed for averaging. There are many random variables in the simulation as currents, connectivity, initial conditions.

            with open(csv_file_path, mode='a', newline='') as file:
                writer = csv.writer(file)
                if file.tell() == 0:
                    writer.writerow(column_names)
                

                path_currents_seed = os.path.join(sub_currents, f'simulation_{num_simulation}_currents_pqif_{pqif}.csv')

                with open(path_currents_seed, mode='a', newline='') as file_:
                    writer_ = csv.writer(file_)
                    if file_.tell() == 0:
                        writer_.writerow(['pqif', 'seed', 'iloop', 'it', 'II_0', 'v_0', 'Inoise_0', 'II_1', 'v_1', 'Inoise_1', 'II_N2+1', 'v_N2+1','Inoise_N2+1', 'II_N2+2', 'v_N2+2', 'Inoise_N2+2'])

                    nqif = int(N * pqif)


                    for seed in range(cant_seed):

                        target, phase, amp, omega, romega_vec, amp0 = generate_target(romega1=1, romega2=5, amp0=amp0)  # moved inside seed loop
                        
                        save_target(seed, target, phase=phase, omega=omega, romega_vec=romega_vec, amp=amp, amp0=amp0, num_simulation=num_simulation, folder_name=nombre_carpeta, pqif=pqif)   # moved inside seed loop


                        np.random.seed(seed = seed)

                        x, r, nspike = initialize_neurons(N)

                        # external current
                        Iext= currents(N, itmax)

                        path_currents_seed = os.path.join(nombre_carpeta, f'simulation_{num_simulation}_Iext_pqif_{pqif}_seed_{seed}.csv')
                        save_matrix_csv(Iext, path_currents_seed)

                        w = initialize_connectivity_matrix(N, p, gsyn)

                        norm_w0 = np.linalg.norm(w)

                        P, idx = initialize_training(N, w)

                        filename_dw = os.path.join(nombre_carpeta, f'simulation_{num_simulation}_dw_pqif_{pqif}_seed_{seed}.csv')

                        with open(filename_dw, mode='w', newline='') as file:
                            csv_writer_dw = csv.writer(file)
                            csv_writer_dw.writerow(['modt', 'modw'])  # Escribir encabezados

                            for iloop in range(nloop):

                                path_inputs= os.path.join(sub_inputs, f'simulation_{num_simulation}_inputs_pqif_{pqif}_iloop_{iloop}_seed_{seed}.csv')
                                path_nspikes = os.path.join(sub_nspikes, f'simulation_{num_simulation}_nspikes_pqif_{pqif}_iloop_{iloop}_seed_{seed}.csv')
                                path_outputs= os.path.join(sub_outputs, f'simulation_{num_simulation}_outputs_pqif_{pqif}_iloop_{iloop}_seed_{seed}.csv')

                                for it in range(itmax):
                                    nspike = np.zeros(N)

                                    x, r, nspike, rout, II, v = evolution(x, r, Iext, w, nqif, it, dt, iout, nspike, b, vt=vt, vrest=vrest)

                                    entrada = II +v

                                    rout_row = rout.reshape(1, -1)
                                    entrada_row = entrada.reshape(1,-1)



                                    if iloop in [nloop_train + 1, nloop - 1] and it % 20 == 0:
                                        writer_.writerow([pqif, seed, iloop, it, II[0], v[0], II[1], v[1], II[N2+1], v[N2+1], II[N2+2], v[N2+2]])

                                    # learning, only in the training loops (0<=iloop<=nloop_train)
                                    if  iloop>0  and iloop <= nloop_train and int(it>itstim):
                                        w, P = learning(it, iloop, w, r, P, idx, target, norm_w0, csv_writer_dw)


                                    nspike_row = np.array(nspike).reshape(1, -1)
                                    

                                    # Save `nspike_row` as a row in the CSV
                                    with open(path_nspikes, 'a') as f:
                                        np.savetxt(f, nspike_row, delimiter=',')

                                    # Save `entrada_row` as a row in the CSV
                                    with open(path_inputs, 'a') as f:
                                        np.savetxt(f, entrada_row, delimiter=',')

                                    # Save `rout_row` as a row in the CSV
                                    with open(path_outputs, 'a') as f:
                                        np.savetxt(f, rout_row, delimiter=',')


                                sigma2,tau_rec,tau_div,tau_con,tau_chn=motifs(w,gsyn,N)
                                


                                if iloop == 0 or iloop == (nloop_train + 1):
                                    path_w_seed = os.path.join(sub_pesos, f'simulation_{num_simulation}_pesos_pqif_{pqif}_matriz_iloop_{iloop}_semilla_{seed}')
                                    save_matrix_csv(w, path_w_seed)



                                writer.writerow([
                                    pqif,
                                    seed,
                                    iloop,
                                    sigma2,
                                    tau_rec,
                                    tau_div,
                                    tau_con,
                                    tau_chn, 
                                        
                                
                                ])